In [ ]:
# instructions:
# download pix3d
# fix models in pix3d which have multiple obj files (no single "model.obj"); rename one of them to be "model.obj"
# For chair class:
# IKEA_JULES_1/model_-0.248241748473_0.131155721985.obj
# IKEA_MARKUS/model_-0.87337964205_-0.0723611270545.obj
# IKEA_PATRIK/model_-0.104274359838_-0.0651591010433.obj
# IKEA_SKRUVSTA/model_0.398123524547_0.0140008021241.obj
# IKEA_SNILLE_1/model_0.622225291388_0.0492453107028.obj

# get branch of occnet for processing
# follow instructions in external (download opengl, glew, etc and build) and set pythonpath
# run bash dataset_pix3d/build.sh
# place pix3d.build into data/external (only 4_pointcloud and 4_points are necessary)

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import json
import pprint
import subprocess
from PIL import Image
import PIL.ImageOps    


In [3]:
PIX3D_DIR = "../data/external/pix3d"
PIX3D_POINTS_DIR = "../data/external/pix3d.build"

OUT_DIR = "../data/pix3d_processed"

classes = ['chair']

In [28]:
with open(os.path.join(PIX3D_DIR, "pix3d.json")) as f:
    pix3d_data_json = json.loads(f.read())
# convert list of dicts into a dict (keyed by image path) of dicts
pix3d_data_dict = { entry["img"]:entry for entry in pix3d_data_json}

In [43]:
# copies files from pix3d and pix3d.build into new pix3d_processed folder.
# images are cropped to have the chair in the center; versions with and without BG are provided;
# points are rotated 90 degrees CCW about y-axis to orient chairs to shapenet (need to check if this is the case for other classes as well)
# https://stackoverflow.com/questions/11142851/adding-borders-to-an-image-using-python
for class_name in classes:
    for root, dirs, files in os.walk(os.path.join(PIX3D_DIR, "img", class_name)):
        for f in files:
            
            # getting paths of assets
            image_name = f.replace(".jpg","").replace(".jpeg", "").replace(".png","").replace(".tiff","")
            image_path = os.path.join(root, f)
            image_dict = pix3d_data_dict[image_path.replace(PIX3D_DIR+'/', "")]
            mask_path = os.path.join(PIX3D_DIR, image_dict['mask'])
            model_name = image_dict['model'].split('/')[2]
            pointcloud_path = os.path.join(PIX3D_POINTS_DIR, class_name, "4_pointcloud", "{}.npz".format(model_name))
            points_path = os.path.join(PIX3D_POINTS_DIR, class_name, "4_points", "{}.npz".format(model_name))
            
            processed_dir = os.path.join(OUT_DIR, class_name, image_name)
            processed_img_dir = os.path.join(processed_dir, "img_pix3d")
            
            if not os.path.exists(processed_img_dir):
                os.makedirs(processed_img_dir)
                
            # ------------------------------------------------------------------------------
            # creating image
            img = Image.open(image_path)
            img_width = image_dict['img_size'][0]
            img_height = image_dict['img_size'][1]
            
            # create expanded version of img
            new_size = (max(img_height, img_width)+100, max(img_height, img_width)+100)
            img_bordered = Image.new("RGB", new_size, (255, 255, 255))
            img_bordered.paste(img, (int((new_size[0]-image_dict['img_size'][0])/2),int((new_size[1]-image_dict['img_size'][1])/2)))
            
            # processing mask
            mask = Image.open(mask_path)
            img_masked = Image.composite(Image.new("RGB", image_dict['img_size'], (255, 255, 255)), img, PIL.ImageOps.invert(mask))
            img_bordered_mask = Image.new("RGB", new_size, (255, 255, 255))
            img_bordered_mask.paste(img_masked, (int((new_size[0]-image_dict['img_size'][0])/2),int((new_size[1]-image_dict['img_size'][1])/2)))
            
            # creating square, centered crop
            bbox_height = image_dict['bbox'][3] - image_dict['bbox'][1]
            bbox_width = image_dict['bbox'][2] - image_dict['bbox'][0]
            
            # adjusting bbox to match after expanding image
            if img_height > img_width:
                img_new_bbox = [image_dict['bbox'][0]+50+(img_height-img_width)/2, image_dict['bbox'][1]+50, image_dict['bbox'][2]+50+(img_height-img_width)/2, image_dict['bbox'][3]+50]
            else:
                img_new_bbox = [image_dict['bbox'][0]+50, image_dict['bbox'][1]+50+(img_width-img_height)/2, image_dict['bbox'][2]+50, image_dict['bbox'][3]+50+(img_width-img_height)/2]
            
            # adjusting bbox so it's a square
            if bbox_height > bbox_width:
                img_new_bbox[0]-=(bbox_height-bbox_width)/2
                img_new_bbox[2]+=(bbox_height-bbox_width)/2
            else:
                img_new_bbox[1]-=(bbox_width-bbox_height)/2
                img_new_bbox[3]+=(bbox_width-bbox_height)/2
                
            # correct possible off-by-one error in image size
            img_new_bbox[0] = int(img_new_bbox[0])
            img_new_bbox[1] = int(img_new_bbox[1])
            img_new_bbox[2] = int(img_new_bbox[2])
            img_new_bbox[3] = int(img_new_bbox[3])
            if img_new_bbox[2] - img_new_bbox[0] > img_new_bbox[3] - img_new_bbox[1]:
                img_new_bbox[2] -= 1
            elif img_new_bbox[2] - img_new_bbox[0] < img_new_bbox[3] - img_new_bbox[1]:
                img_new_bbox[3] -= 1
            if img_new_bbox[2] - img_new_bbox[0] != img_new_bbox[3] - img_new_bbox[1]:
                print("final image not square")
                raise 
                
            img_bordered = img_bordered.crop(img_new_bbox)
            img_bordered.save(os.path.join(processed_img_dir,'bg.jpg'))
            img_bordered_mask = img_bordered_mask.crop(img_new_bbox)
            img_bordered_mask.save(os.path.join(processed_img_dir,'no_bg.jpg'))
            

            # ------------------------------------------------------------------------------
            # creating points/pointclouds
            # y-axis ccw by 90 deg
            rot_mat = np.array([[0,0,1],[0,1,0],[-1,0,0]])
            pc = np.load(pointcloud_path)
            np.savez(os.path.join(processed_dir,'pointcloud.npz'), points=pc["points"] @ rot_mat,
                     normals=pc["normals"] @ rot_mat, loc=pc["loc"], scale=pc["scale"])
            
            p = np.load(points_path)
            np.savez(os.path.join(processed_dir,'points.npz'), points=p["points"] @ rot_mat,
                     occupancies=p["occupancies"], loc=p["loc"], scale=p["scale"])  
            
            

In [45]:
# create split lsts. For pix3d, it's only a test set
for class_name in classes:
    class_img_codes = [ code.split('/')[2][:4] for code in list(pix3d_data_dict.keys()) if class_name in code ]
    with open (os.path.join(OUT_DIR, class_name, "train.lst"), 'w') as f:
        pass
    with open (os.path.join(OUT_DIR, class_name, "val.lst"), 'w') as f:
        pass
    with open (os.path.join(OUT_DIR, class_name, "test.lst"), 'w') as f:
        for i, img_code in enumerate(class_img_codes):
            if i == len(class_img_codes)-1:
                f.write("{}".format(img_code))
            else:
                f.write("{}\n".format(img_code))

In [21]:
# make uda target domain dataset folder by copying over all images
TARGET_DOMAIN_OUT_DIR_NO_BG = "../data/target_domain/no_bg"
TARGET_DOMAIN_OUT_DIR_BG = "../data/target_domain/bg"
  
for class_name in classes:
    # TODO: need to convert class_name to shapenet conventions
    if not os.path.exists(os.path.join(TARGET_DOMAIN_OUT_DIR_BG, class_name)):
        os.makedirs(os.path.join(TARGET_DOMAIN_OUT_DIR_BG, class_name))
    if not os.path.exists(os.path.join(TARGET_DOMAIN_OUT_DIR_NO_BG, class_name)):
        os.makedirs(os.path.join(TARGET_DOMAIN_OUT_DIR_NO_BG, class_name))
    for root, dirs, files in os.walk(os.path.join(OUT_DIR, class_name)):
        for f in files:
            in_path = os.path.join(root, f)
            obj_id = in_path.split('/')[-3]

            if "no_bg.jpg" in in_path:
                out_path = os.path.join(TARGET_DOMAIN_OUT_DIR_NO_BG, class_name, "pix3d_{}.jpg".format(obj_id))
                print(in_path, out_path)
                subprocess.run(['cp', in_path, out_path], stdout = subprocess.PIPE, stderr=subprocess.PIPE, check=True, universal_newlines=True)
            elif "bg.jpg" in in_path:
                out_path = os.path.join(TARGET_DOMAIN_OUT_DIR_BG, class_name, "pix3d_{}.jpg".format(obj_id))
                print(in_path, out_path)
                subprocess.run(['cp', in_path, out_path], stdout = subprocess.PIPE, stderr=subprocess.PIPE, check=True, universal_newlines=True)

../data/pix3d_processed/chair/2861/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2861.jpg
../data/pix3d_processed/chair/2861/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2861.jpg
../data/pix3d_processed/chair/0641/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0641.jpg
../data/pix3d_processed/chair/0641/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0641.jpg
../data/pix3d_processed/chair/2914/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2914.jpg
../data/pix3d_processed/chair/2914/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2914.jpg
../data/pix3d_processed/chair/0317/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0317.jpg
../data/pix3d_processed/chair/0317/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0317.jpg
../data/pix3d_processed/chair/1907/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1907.jpg
../data/pix3d_processed/chair/1907/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d

../data/pix3d_processed/chair/0877/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0877.jpg
../data/pix3d_processed/chair/0877/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0877.jpg
../data/pix3d_processed/chair/0236/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0236.jpg
../data/pix3d_processed/chair/0236/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0236.jpg
../data/pix3d_processed/chair/1321/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1321.jpg
../data/pix3d_processed/chair/1321/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1321.jpg
../data/pix3d_processed/chair/1395/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1395.jpg
../data/pix3d_processed/chair/1395/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1395.jpg
../data/pix3d_processed/chair/3454/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_3454.jpg
../data/pix3d_processed/chair/3454/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d

../data/pix3d_processed/chair/1936/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1936.jpg
../data/pix3d_processed/chair/1936/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1936.jpg
../data/pix3d_processed/chair/0332/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0332.jpg
../data/pix3d_processed/chair/0332/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0332.jpg
../data/pix3d_processed/chair/1864/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1864.jpg
../data/pix3d_processed/chair/1864/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1864.jpg
../data/pix3d_processed/chair/2310/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2310.jpg
../data/pix3d_processed/chair/2310/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2310.jpg
../data/pix3d_processed/chair/0940/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0940.jpg
../data/pix3d_processed/chair/0940/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d

../data/pix3d_processed/chair/2918/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2918.jpg
../data/pix3d_processed/chair/2918/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2918.jpg
../data/pix3d_processed/chair/2744/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2744.jpg
../data/pix3d_processed/chair/2744/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2744.jpg
../data/pix3d_processed/chair/3628/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_3628.jpg
../data/pix3d_processed/chair/3628/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_3628.jpg
../data/pix3d_processed/chair/0266/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0266.jpg
../data/pix3d_processed/chair/0266/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0266.jpg
../data/pix3d_processed/chair/3551/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_3551.jpg
../data/pix3d_processed/chair/3551/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d

../data/pix3d_processed/chair/0789/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0789.jpg
../data/pix3d_processed/chair/3785/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_3785.jpg
../data/pix3d_processed/chair/3785/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_3785.jpg
../data/pix3d_processed/chair/2773/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2773.jpg
../data/pix3d_processed/chair/2773/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2773.jpg
../data/pix3d_processed/chair/2407/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2407.jpg
../data/pix3d_processed/chair/2407/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2407.jpg
../data/pix3d_processed/chair/3140/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_3140.jpg
../data/pix3d_processed/chair/3140/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_3140.jpg
../data/pix3d_processed/chair/1151/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d

...

../data/pix3d_processed/chair/2022/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2022.jpg
../data/pix3d_processed/chair/2022/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2022.jpg
../data/pix3d_processed/chair/0855/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0855.jpg
../data/pix3d_processed/chair/0855/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0855.jpg
../data/pix3d_processed/chair/3714/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_3714.jpg
../data/pix3d_processed/chair/3714/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_3714.jpg
../data/pix3d_processed/chair/2753/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2753.jpg
../data/pix3d_processed/chair/2753/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2753.jpg
../data/pix3d_processed/chair/0360/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0360.jpg
../data/pix3d_processed/chair/0360/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d

../data/pix3d_processed/chair/2535/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2535.jpg
../data/pix3d_processed/chair/0045/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0045.jpg
../data/pix3d_processed/chair/0045/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0045.jpg
../data/pix3d_processed/chair/1497/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1497.jpg
../data/pix3d_processed/chair/1497/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1497.jpg
../data/pix3d_processed/chair/2203/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2203.jpg
../data/pix3d_processed/chair/2203/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2203.jpg
../data/pix3d_processed/chair/2874/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2874.jpg
../data/pix3d_processed/chair/2874/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2874.jpg
../data/pix3d_processed/chair/3608/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d

../data/pix3d_processed/chair/3590/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_3590.jpg
../data/pix3d_processed/chair/3590/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_3590.jpg
../data/pix3d_processed/chair/1109/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1109.jpg
../data/pix3d_processed/chair/1109/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1109.jpg
../data/pix3d_processed/chair/1358/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1358.jpg
../data/pix3d_processed/chair/1358/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1358.jpg
../data/pix3d_processed/chair/2735/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2735.jpg
../data/pix3d_processed/chair/2735/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2735.jpg
../data/pix3d_processed/chair/0584/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0584.jpg
../data/pix3d_processed/chair/0584/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d

../data/pix3d_processed/chair/0736/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0736.jpg
../data/pix3d_processed/chair/0736/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0736.jpg
../data/pix3d_processed/chair/0852/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0852.jpg
../data/pix3d_processed/chair/0852/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0852.jpg
../data/pix3d_processed/chair/1129/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1129.jpg
../data/pix3d_processed/chair/1129/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1129.jpg
../data/pix3d_processed/chair/1759/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1759.jpg
../data/pix3d_processed/chair/1759/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1759.jpg
../data/pix3d_processed/chair/2247/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2247.jpg
../data/pix3d_processed/chair/2247/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d

../data/pix3d_processed/chair/1963/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1963.jpg
../data/pix3d_processed/chair/1916/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1916.jpg
../data/pix3d_processed/chair/1916/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1916.jpg
../data/pix3d_processed/chair/0808/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_0808.jpg
../data/pix3d_processed/chair/0808/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_0808.jpg
../data/pix3d_processed/chair/1711/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_1711.jpg
../data/pix3d_processed/chair/1711/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_1711.jpg
../data/pix3d_processed/chair/2782/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d_2782.jpg
../data/pix3d_processed/chair/2782/img_pix3d/no_bg.jpg ../data/target_domain/no_bg/chair/pix3d_2782.jpg
../data/pix3d_processed/chair/2515/img_pix3d/bg.jpg ../data/target_domain/bg/chair/pix3d